<a href="https://colab.research.google.com/github/Kimeteor309/Financial-big-data-analysis/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90_week13%E4%BD%9C%E6%A5%AD_%E5%9F%BA%E5%9B%A0%E6%BC%94%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance numpy

In [2]:
import yfinance as yf
import numpy as np

# 股票池
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "JPM", "V", "JNJ"] # 使用更多股票（例如加入科技、醫療、金融等不同行業的標的）來測試

# 獲取歷史價格
def get_stock_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    return data

# 計算日回報率和其期望值、風險
def calculate_returns_and_risks(data):
    returns = data.pct_change().dropna()
    mean_returns = returns.mean()  # 平均回報率
    risks = returns.std()  # 標準差作為風險指標
    return mean_returns.values, risks.values

# 獲取股票數據
start_date = "2023-01-01"
end_date = "2024-01-01"
data = get_stock_data(tickers, start_date, end_date)
mean_returns, risks = calculate_returns_and_risks(data)


[*********************100%***********************]  10 of 10 completed


In [10]:
# 基因演算法參數
population_size = 50
generations = 100
mutation_rate = 0.05 # 增加突變率
num_stocks = len(tickers)

# 新的適應函數
def fitness(chromosome):
    selected_returns = mean_returns[chromosome == 1]
    selected_risks = risks[chromosome == 1]
    num_selected = np.sum(chromosome) # 選股數量
    max_stocks = 5  # 最大持股數量
    if len(selected_returns) == 0 or num_selected > max_stocks or np.mean(selected_returns) < 0.002: # 避免空組合或超過限制、設定回報下限
        return 0  # 返回最低適應值，而不是負值
    return np.mean(selected_returns) - 0.7 * np.mean(selected_risks) + 0.1 * num_selected # 增加多樣性懲罰，鼓勵選擇更多股票；目前風險權重0.5，可以更改成0.7(保守型)或0.3(機進型)

# 初始化族群
population = np.random.choice([0, 1], size=(population_size, num_stocks), p=[0.5, 0.5]) # 初始化族群：隨機產生多樣化初始族群

# 基因演算法主迴圈
for generation in range(generations):
    fitness_scores = np.array([fitness(ind) for ind in population])

    # 選擇
    probabilities = fitness_scores / fitness_scores.sum()
    selected_indices = np.random.choice(range(population_size), size=population_size, p=probabilities)
    selected_population = population[selected_indices]

    # 交配
    new_population = []
    for i in range(0, population_size, 2):
        parent1, parent2 = selected_population[i], selected_population[i + 1]
        crossover_point = np.random.randint(1, num_stocks - 1)
        child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
        new_population.extend([child1, child2])

    # 突變
    for individual in new_population:
        if np.random.rand() < mutation_rate:
            mutation_point = np.random.randint(0, num_stocks)
            individual[mutation_point] = 1 - individual[mutation_point]

    population = np.array(new_population)

# 找出最佳解
best_chromosome = population[np.argmax([fitness(ind) for ind in population])]
selected_stocks = [tickers[i] for i, selected in enumerate(best_chromosome) if selected == 1]
best_return = np.mean(mean_returns[best_chromosome == 1])
best_risk = np.mean(risks[best_chromosome == 1])

print("最佳投資組合：", selected_stocks)
print("期望回報率：", best_return)
print("風險：", best_risk)


最佳投資組合： ['AAPL', 'TSLA', 'META', 'NVDA', 'JPM']
期望回報率： 0.0029739711269423435
風險： 0.019419434632817967
